In [1]:
import os, sys, json
from xml.etree.ElementTree import parse
from urllib.parse import urlencode, unquote, quote_plus
import xmltodict
import pandas as pd

if sys.version_info[0]==3:
    from urllib.request import urlopen
else:
    from urllib import urlopen


BASE_DIR = './'

# Secret File Load
secret_file = os.path.join(BASE_DIR,'secret.json')
with open(secret_file) as f:
    secrets = json.loads(f.read())

KEY = unquote(secrets['Enecoding'])

In [3]:
# to get the period data
preiod = pd.read_csv('data/api_tradePrc.csv')
list_preiod = preiod.columns
list_preiod = list_preiod[4:]

In [4]:
def call_open_api(call_code, yyyymm) : 

    buliding_type = call_code.split('c')[1][:3]
    #different server by buliding type
    server = ':8081' if buliding_type == 'Apt' else ''
    URL = 'http://openapi.molit.go.kr{}'.format(server)+'/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/{}'.format(call_code)

    
    queryParams = '?' + urlencode(
        {
            quote_plus('serviceKey') : KEY,
            quote_plus('LAWD_CD') : '36110',
            quote_plus('DEAL_YMD') : yyyymm,

        }
    )

    response = urlopen(URL + queryParams).read()
    decode_data = response.decode('utf-8')
    
    xml_parse =  xmltodict.parse(decode_data)
    xml_dict = json.loads(json.dumps(xml_parse))

    return xml_dict

In [5]:
def writeDatafram(call_code, yyyymm) :

    api_data = call_open_api(call_code, yyyymm)
    new_data = api_data['response']['body']['items']['item']

    total = api_data['response']['body']['totalCount']
    df = pd.DataFrame.from_dict(new_data, orient= 'index').transpose() if total == '1' else pd.DataFrame(new_data)
 
    return df

### apt

In [6]:
apt_data_rent = pd.DataFrame()
for i in list_preiod:

    df = writeDatafram('getRTMSDataSvcAptRent',i)
    apt_data_rent = apt_data_rent.append(df, ignore_index=True)

display(apt_data_rent.tail())

,건축년도,년,법정동,보증금액,아파트,월,월세금액,일,전용면적,지번,지역코드,층
40708,2000,2021,금남면 용포리,"1,000",두진리버빌,2,40,5,43.008,320,36110,3
40709,2000,2021,금남면 용포리,500,두진리버빌,2,40,9,43.008,320,36110,3
40710,2000,2021,금남면 용포리,"22,000",두진리버빌,2,0,15,59.8676,320,36110,1
40711,2001,2021,전의면 유천리,"1,000",민석그린,2,27,16,59.94,528,36110,1
40712,2001,2021,전의면 유천리,"4,500",민석그린,2,0,23,59.94,528,36110,1


In [14]:
apt_data_rent.to_csv('data/아파트_전월세.csv',encoding='utf-8-sig')

In [8]:
api_data_trd = pd.DataFrame()
for i in list_preiod:

    df = writeDatafram('getRTMSDataSvcAptTrade',i)

    api_data_trd = api_data_trd.append(df, ignore_index=True)

api_data_trd.tail()

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
24412,"7,000",2001,2021,전의면 유천리,민석그린,2,18,59.94,528,36110,5,None,None
24413,"5,200",1994,2021,전동면 노장리,도원,2,3,49.32,586,36110,5,None,None
24414,"5,200",1994,2021,전동면 노장리,도원,2,23,49.32,586,36110,5,None,None
24415,"5,000",1996,2021,소정면 운당리,무궁화,2,1,59.4,35-1,36110,7,None,None
24416,"6,300",1996,2021,소정면 운당리,무궁화,2,8,59.4,35-1,36110,7,None,None


In [15]:
api_data_trd.to_csv('data/아파트_매매.csv',encoding='utf-8-sig')

### ofi

In [23]:
ofi_data_rent = pd.DataFrame()
for i in list_preiod :

    df = writeDatafram('getRTMSDataSvcOffiRent',i)

    ofi_data_rent = ofi_data_rent.append(df, ignore_index=True)

ofi_data_rent.tail()

,건축년도,년,단지,법정동,보증금,월,월세,일,전용면적,지역코드,층,지번
1292,2014,2021,세종한신휴시티,도담동,500,2,45,11,24.03,36110,4,246-5
1293,2014,2021,세종한신휴시티,도담동,"4,000",2,25,17,27.43,36110,5,246-5
1294,2014,2021,세종한신휴시티,도담동,500,2,45,20,24.03,36110,8,246-5
1295,2019,2021,주건축물제1동,연동면 응암리,"8,000",2,0,8,43,36110,3,911
1296,2019,2021,주건축물제1동,연동면 응암리,"8,000",2,0,8,41.29,36110,4,911


In [203]:
ofi_data_trd = pd.DataFrame()
for i in list_preiod:

    df = writeDatafram('getRTMSDataSvcOffiTrade',i)
    ofi_data_trd = ofi_data_trd.append(df, ignore_index=True)

ofi_data_trd.tail()

,거래금액,건축년도,년,단지,법정동,월,일,전용면적,지역코드,층,해제사유발생일,해제여부,지번
368,"8,500",2014,2021,푸르지오시티,어진동,2,18,23.74,36110,5,21.03.09,O,NaN
369,"8,500",2014,2021,푸르지오시티,어진동,2,18,23.74,36110,5,None,None,NaN
370,"9,500",2014,2021,푸르지오시티,어진동,2,19,23.74,36110,17,None,None,NaN
371,"8,650",2014,2021,푸르지오시티,어진동,2,25,24.12,36110,6,None,None,NaN
372,"8,100",2014,2021,2차푸르지오시티,어진동,2,27,21.92,36110,7,None,None,NaN
